### Project Phase 1
In this phase, we:
1. implement Tree Structure
2. implement 2 vaccinate algorithm: degree-heavy and subtree-heavy

Note:
* The tree node is implemented with a struct Node
* The tree is represented by a dict.
* Each node is map to a Node object in the nodes list

In [6]:
# tree node
class Node:
    def __init__(self, index):
        self.index = index
        self.subtree_count = 0

class Simulation:
    def __init__(self):
        self.nodes = []
        self.tree = {}
        self.F = set()
        self.P = set()
        self.count = 0

    def __del__(self):
        print("end simulation")

    # construct the tee
    def construct_tree(self, file_name):
        def read_data(file_name):
            with open(file_name) as f:
                lines = f.readlines()
            str_list_lines = [line.rstrip("\n").rstrip(" ").lstrip(" ").split(" ") for line in lines]
            int_list_lines = []
            for str_list in str_list_lines:
                try:
                    int_list_lines.append(list(map(int, str_list)))
                except:
                    raise Exception("Invalid input format!")
            return int_list_lines
        int_list_lines = read_data(file_name)
        num_nodes = int_list_lines[0][0]
        self.count = num_nodes
        # map node with index of node
        for i in range(num_nodes+1):
            self.nodes.append(Node(i))
        # create a tree structure (in which all the node is represented with integer)
        self.tree = dict.fromkeys([i+1 for i in range(num_nodes)])
        for int_list in int_list_lines[1:]:
            if len(int_list) == 1:
                self.tree[int_list[0]] = None # leaf of the tree
            else:
                self.tree[int_list[0]] = int_list[1:]

    # update the subtree_count of all the nodes in the tree
    def update_subtree_count_tree(self):
        def update_subtree_count(node_index):
            if self.tree[node_index] is None:
                self.nodes[node_index].subtree_count = 0
                return 0
            current_subtree_count = 0
            for child_index in self.tree[node_index]:
                current_subtree_count += 1
                current_subtree_count += update_subtree_count(child_index)
            self.nodes[node_index].subtree_count = current_subtree_count
            return current_subtree_count
        update_subtree_count(1)

    # infection
    def infect_round(self, F_last):
        """
        :param F_last: (set(int)) infected node indexes of last iteration
        :return F_new: (set(int)) newly infected node indexes
        """
        F_new = set() # newly infected nodes
        for index in F_last:
            if self.tree[index] is not None:
                for child in self.tree[index]:
                    if child not in self.P:
                        F_new.add(child)
        self.F = self.F.union(F_new)
        return F_new

    # vaccinate
    def vaccinate(self, node_index):
        # we cannot vaccinate node which is already in P
        self.P.add(node_index)

    # degree_heavy_implementation
    def degree_heavy(self, infected_indexes):
        """
        :param infected_indexes: (int) the current infected
        :return: vac_index: (int) the index of the node to be vacinated
        """
        max_child_num = 0
        max_child_index = -1
        for node_index in infected_indexes:
            if self.tree[node_index] is None:
                continue
            for neighbor_index in self.tree[node_index]:
                if neighbor_index not in self.P and neighbor_index not in self.F:
                    if self.tree[neighbor_index] is not None:
                        current_child_num = len(self.tree[neighbor_index])
                        if current_child_num > max_child_num:
                            max_child_num = current_child_num
                            max_child_index = neighbor_index
        if max_child_index == -1:
            return None
        self.P.add(max_child_index)
        return max_child_index

    def subtree_heavy(self, infected_indexes):
        """
        :param infected_indexes: (int) the current infected
        :return: vac_index: (int) the index of the node to be vacinated
        """
        max_child_num = 0
        max_child_index = -1
        for node_index in infected_indexes:
            if self.tree[node_index] is None:
                continue
            for neighbor_index in self.tree[node_index]:
                if neighbor_index not in self.P and neighbor_index not in self.F:
                    if self.tree[neighbor_index] is not None:
                        current_child_num = self.nodes[neighbor_index].subtree_count
                        if current_child_num > max_child_num:
                            max_child_num = current_child_num
                            max_child_index = neighbor_index
        if max_child_index == -1:
            return None
        self.P.add(max_child_index)
        return max_child_index

    # simulate: degree_heavy
    def degree_heavy_simulate(self, input_file):
        self.construct_tree(input_file)
        self.update_subtree_count_tree()
        F_new = set()
        F_new.add(1)
        self.F.add(1)
        while len(F_new) > 0:
            vac_index = self.degree_heavy(F_new)
            F_new = self.infect_round(F_new)
        print(len(self.F))
        print(self.count)
        print(len(self.F) / self.count)

    # simulate: subtree_heavy
    def subtree_heavy_simulate(self, input_file):
        self.construct_tree(input_file)
        self.update_subtree_count_tree()
        F_new = set()
        F_new.add(1)
        self.F.add(1)
        while len(F_new) > 0:
            vac_index = self.subtree_heavy(F_new)
            F_new = self.infect_round(F_new)
        print(len(self.F))
        print(self.count)
        print(len(self.F) / self.count)

if __name__ == '__main__':
    simulation = Simulation()
    simulation.subtree_heavy_simulate("testCase10.txt")
    simulation.__del__()

end simulation
76736
353695
0.21695528633427105
end simulation
